In [53]:
import pandas as pd
import os
import json
import numpy as np
from collections import defaultdict
import json
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import requests

## Some preprocessing code I'll include: not super relevant, but it's useful.

**(goes from a bunch of txt files to csv)**

In [38]:
files = [
    "kanye.txt",
    "eminem.txt",
    "denzel.txt",
    "BIG.txt",
    "biggie cheese.txt",
    "tupac.txt"
]

file_dict = defaultdict(list)

for file_name in files:
    with open(file_name, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines]
    lines = [x for x in lines if x]
    file_dict[file_name] = lines

In [39]:
for k in file_dict:
    print(len(file_dict[k]))

134
221
52
93
136
183


In [40]:
chunked_dict = defaultdict(list)
for k in file_dict:
    text = []
    for curtext in file_dict[k]:
        curlen = len(curtext.split())
        if curlen > 255:
            print("FAT")
        curdenom = 1 + ((curlen)//256)
        if len(curtext.split())%256 == 0:
            curdenom = curdenom - 1
        for j in range(curdenom):
            text.append(" ".join(curtext.split()[j*(curlen//curdenom):(j+1)*(curlen//curdenom)]))
    chunked_dict[k] = text

FAT
FAT
FAT
FAT


In [41]:
cnt = 0
for k in file_dict:
    print(len(file_dict[k]))
    print(len(chunked_dict[k]))
    print(k)
    cnt+=len(chunked_dict[k])
#ait it works

134
134
kanye.txt
221
225
eminem.txt
52
52
denzel.txt
93
93
BIG.txt
136
136
biggie cheese.txt
183
183
tupac.txt


In [48]:
person = []
text = []
fiction_or_not = []
for k in file_dict:
    for ele in file_dict[k]:
        if k in ["kanye.txt", "eminem.txt", "denzel.txt", "BIG.txt", "tupac.txt"]:
            fiction_or_not.append("True")
        else:
            fiction_or_not.append("False")
        person.append(k)
        text.append(ele)
vkg_df = pd.DataFrame(data = {"text": text, "person": person, "fiction_or_not": fiction_or_not})
vkg_df.to_csv("rapper_vkg.csv", index=False)

## csv2s3 Core Logic

In [65]:
df= pd.read_csv("rapper_vkg.csv")

In [67]:
df = df[df['text'].apply(lambda x: len(x.split()) > 20)] #just some filtering from my end (data was not clean)

In [68]:
len(df)

479

In [189]:
os.environ['AWS_ACCESS_KEY_ID'] = 'key1'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'key2'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
!aws sts get-caller-identity


An error occurred (InvalidClientTokenId) when calling the GetCallerIdentity operation: The security token included in the request is invalid.


In [55]:
s3 = boto3.client('s3')

In [109]:
def create_bucket(bucket_name, region=None):
    try:
        if region:
            s3.create_bucket(
                Bucket=bucket_name,
                # CreateBucketConfiguration={'LocationConstraint': region}
            )
        else:
            s3.create_bucket(Bucket=bucket_name)
        print(f'Bucket {bucket_name} created successfully.')
    except s3.exceptions.BucketAlreadyExists:
        print(f'Bucket {bucket_name} already exists.')
    except s3.exceptions.BucketAlreadyOwnedByYou:
        print(f'Bucket {bucket_name} is already owned by you.')
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')

def list_buckets():
    try:
        response = s3.list_buckets()
        buckets = [{'Name': bucket['Name'], 'CreationDate': bucket['CreationDate']} for bucket in response['Buckets']]
        return buckets
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')
        return []
    except ClientError as e:
        print(f'ClientError: {e}')
        return []

def upload_file(file_name, bucket_name, object_name=None):
    if object_name is None:
        object_name = file_name
    try:
        s3.upload_file(file_name, bucket_name, object_name)
        print(f'File {file_name} uploaded to bucket {bucket_name} as {object_name}.')
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')

# Function to download a file from a bucket
def download_file(bucket_name, object_name, file_name):
    try:
        s3.download_file(bucket_name, object_name, file_name)
        print(f'File {file_name} downloaded from bucket {bucket_name} as {object_name}.')
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')

# Function to delete a file from a bucket
def delete_file(bucket_name, object_name):
    try:
        s3.delete_object(Bucket=bucket_name, Key=object_name)
        print(f'File {object_name} deleted from bucket {bucket_name}.')
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')

# Function to delete a bucket
def delete_bucket(bucket_name):
    try:
        s3.delete_bucket(Bucket=bucket_name)
        print(f'Bucket {bucket_name} deleted successfully.')
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')

def list_files(bucket_name):
    s3 = boto3.client('s3')
    file_names = []
    response = s3.list_objects_v2(Bucket=bucket_name)
    if 'Contents' in response:
        for obj in response['Contents']:
            file_names.append(obj['Key'])
    
    return file_names

def upload_dict_to_s3(bucket_name, object_name, dictionary):
    json_data = json.dumps(dictionary)
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=json_data)

def download_file_to_dict(bucket_name, object_name):
    try:
        response = s3.get_object(Bucket=bucket_name, Key=object_name)
        content = response['Body'].read().decode('utf-8')
        data_dict = json.loads(content)
        # print(f'File {object_name} downloaded from bucket {bucket_name} and converted to dictionary.')
        return data_dict
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')
    except Exception as e:
        print(f'An error occurred: {e}')
        return None

def empty_bucket(bucket_name):
    try:
        s3 = boto3.client('s3')
        s3_resource = boto3.resource('s3')
        bucket = s3_resource.Bucket(bucket_name)

        # Delete all objects in the bucket
        bucket.objects.delete()


        print(f'Bucket {bucket_name} has been emptied.')
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')
    except Exception as e:
        print(f'An error occurred: {e}')

In [185]:
region = 'us-east-1'
bucket_name = "rapper-vkg"
num_batches = 7
for i in range(num_batches):
    create_bucket(bucket_name+f"-{i}", region)

Bucket rapper-vkg-0 created successfully.
Bucket rapper-vkg-1 created successfully.
Bucket rapper-vkg-2 created successfully.
Bucket rapper-vkg-3 created successfully.
Bucket rapper-vkg-4 created successfully.
Bucket rapper-vkg-5 created successfully.
Bucket rapper-vkg-6 created successfully.


In [186]:
for idx in range(len(df)):
    if idx%10 == 0:
        print(idx)
    modnum = idx%num_batches
    upload_dict_to_s3(bucket_name+f"-{modnum}", f"data_{idx}.json", df.iloc[idx].to_dict())

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470


In [187]:
task_queues = defaultdict(list)
for idx in range(len(df)):    
    modnum = idx%num_batches
    task_queues[modnum].append(f"data_{idx}.json")
for i in range(num_batches):
    print(len(task_queues[i]))

69
69
69
68
68
68
68


In [188]:
for i in range(num_batches):
    td = {
        "task_queue": task_queues[i]
    }
    upload_dict_to_s3(bucket_name+f"-{i}", f"task_queue_{i}.json", td)
print("ALL OK")

ALL OK


In [ ]:
#sample code- just prevents annoying copy pasting:

In [178]:
print(len(df))

479


In [184]:
#DON'T RUN THIS: JUST FOR MY TESTING
for i in range(num_batches):
    empty_bucket(f"rapper-vkg-{i}")
    empty_bucket(f"rapper-vkg-{i}-results")
    delete_bucket(f"rapper-vkg-{i}")
    delete_bucket(f"rapper-vkg-{i}-results")

Bucket rapper-vkg-0 has been emptied.
Bucket rapper-vkg-0-results has been emptied.
Bucket rapper-vkg-0 deleted successfully.
Bucket rapper-vkg-0-results deleted successfully.
Bucket rapper-vkg-1 has been emptied.
Bucket rapper-vkg-1-results has been emptied.
Bucket rapper-vkg-1 deleted successfully.
Bucket rapper-vkg-1-results deleted successfully.
Bucket rapper-vkg-2 has been emptied.
Bucket rapper-vkg-2-results has been emptied.
Bucket rapper-vkg-2 deleted successfully.
Bucket rapper-vkg-2-results deleted successfully.
Bucket rapper-vkg-3 has been emptied.
Bucket rapper-vkg-3-results has been emptied.
Bucket rapper-vkg-3 deleted successfully.
Bucket rapper-vkg-3-results deleted successfully.
Bucket rapper-vkg-4 has been emptied.
Bucket rapper-vkg-4-results has been emptied.
Bucket rapper-vkg-4 deleted successfully.
Bucket rapper-vkg-4-results deleted successfully.
Bucket rapper-vkg-5 has been emptied.
Bucket rapper-vkg-5-results has been emptied.
Bucket rapper-vkg-5 deleted successf